In [2]:
# imports
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [9]:
cid = 'Insert cid token'
secret = 'Insert secret token'

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
playlists_lst = [['Feel Good Dinner','spotify','37i9dQZF1DXbm6HfkbMtFZ'],
                  ['Beast Mode','spotify','37i9dQZF1DX76Wlfdnj7AP'],
                  ['Brain Food','spotify','37i9dQZF1DWXLeA8Omikj7'],
                  ['Top Gaming Tracks','spotify','37i9dQZF1DWTyiBJ6yEqeu'],
                  ["Feelin' Good",'spotify','37i9dQZF1DX9XIFQuFvzM4'],
                  ['Sleep','spotify','37i9dQZF1DWZd79rJ6a7lp'],
                  ['Get Turnt','spotify','37i9dQZF1DWY4xHQp97fN6']]

In [6]:
def playlist_df_generator(playlists_lst):
    '''
    Returns a df filled with consolidated playlist data

    Parameters
    -----------
    playlists_lst: list of lists with each list representing a playlist.
    The pieces of information in each list are:
    1) playlist name
    2) playlist creator's user id
    3) playlist URI
    
    Returns
    --------
    dataframe: filled with playlist data from all provided playlists
    '''
    # Specify the dimensions to save from each track
    playlist_dimensions = ["artist","album","track_name",
                              "track_id","danceability","energy",
                              "key","loudness","mode",
                              "speechiness","instrumentalness",
                              "liveness","valence","tempo",
                              "duration_ms","time_signature"]
    
    # Loop through each playlist
    for i, lst in enumerate(playlists_lst):
        playlist_df= pd.DataFrame(columns=playlist_dimensions)
        # Specify playlist
        playlist = sp.user_playlist_tracks(user=lst[1], playlist_id=lst[2],limit=100)["items"]
        
        #Loop through each track and gather relevant fields
        for track in playlist:
            playlist_features = {}
            playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
            playlist_features["album"] = track["track"]["album"]["name"]
            playlist_features["track_name"] = track["track"]["name"]
            playlist_features["track_id"] = track["track"]["id"]
        
            audio_features = sp.audio_features(playlist_features["track_id"])[0]
            for feature in playlist_dimensions[4:]:
                playlist_features[feature] = audio_features[feature]
        
            # Concat the dfs
            track_df = pd.DataFrame(playlist_features, index = [0])
            playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = lst[0]
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

In [7]:
df = playlist_df_generator(playlists_lst=playlists_lst)

In [8]:
df.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
0,Yumi Zouma,Depths (Pt. I),Depths (Pt. I),7v8B1M09vnvymSNGcvGBva,0.672,0.468,6,-9.570,1,0.0311,0.0084,0.0983,0.645,116.029,170028,4,Feel Good Dinner
1,The California Honeydrops,A River's Invitation,Crazy Girls,78VX1YygxZMqYPGEwMIxBG,0.689,0.630,10,-6.910,1,0.0749,0,0.4630,0.916,113.682,237547,4,Feel Good Dinner
2,Aretha Franklin,A Bit Of Soul,One Step Ahead - Mono Mix,4lH451AAXQD0s8r4mIOsRw,0.587,0.380,0,-8.912,0,0.0251,8.76e-06,0.3930,0.638,85.214,146693,4,Feel Good Dinner
3,Delv!s,No Ending,No Ending,1wId84Uc14XIEIKfcSYAE0,0.710,0.871,2,-4.724,0,0.1940,0.000159,0.3880,0.636,171.993,233907,4,Feel Good Dinner
4,Caamp,Officer of Love,Officer of Love,4KhBvLbRr58rHPF24bdL9Q,0.662,0.574,4,-8.696,1,0.0261,0.00902,0.1150,0.948,109.085,239805,4,Feel Good Dinner
